In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import bentoml
NAME = 'linear_sleep'

# load data & train model(not used, just placeholder)

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.01 0.   1.01 1.55 0.   1.26 1.   0.   1.01 2.   1.01 0.   2.
 0.   1.85 2.   2.   0.   0.   1.   2.   1.   1.35 1.52 1.85 1.   1.
 2.   2.  ]
  mse: 0.088947


In [4]:
%%writefile {NAME}.py

import bentoml
import time
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import DataframeHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):
    @bentoml.api(DataframeHandler, mb_max_latency=10000)
    def predict(self, inputs):
        a, b = inputs.to_numpy()[0]
        x = inputs.shape[0]
        time.sleep(a * x + b)
        return inputs

Overwriting linear_sleep.py


In [5]:
from linear_sleep import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()

[2020-05-13 14:17:00,556] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-13 14:17:00,895] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.4.9+224.g4061229.dirty
creating BentoML-0.4.9+224.g4061229.dirty/BentoML.egg-info
creating BentoML-0.4.9+224.g4061229.dirty/bentoml
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/artifact
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/bundler
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/cli
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/clipper
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/configuration
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/configuration/__pycache__
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/deployment
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/deployment/sagemaker
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/handlers
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/marshal
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/migrations
creating BentoML-0.4.9+224.g406

In [6]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)
print(f"bentoml serve-gunicorn {saved_path} --port {PORT} --workers 1 --enable-microbatch")
#!bentoml serve-gunicorn {saved_path} --port {PORT} --workers 1 --enable-microbatch
server_url = f'http://127.0.0.1:{PORT}/predict'
print(server_url)

51793
bentoml serve-gunicorn /root/bentoml/repository/BentoSvc/20200513141700_DD6843 --port 51793 --workers 1 --enable-microbatch
http://127.0.0.1:51793/predict


# Build & Run Bento Service in Docker

In [7]:
!cd {saved_path}
IMG_NAME = saved_path.split('/')[-1].lower()
!docker build -t {IMG_NAME} {saved_path}
!docker run -itd -p {PORT}:5000 --cpus 1 -e FLAGS="--workers 1 --enable-microbatch" {IMG_NAME}:latest


Sending build context to Docker daemon  594.9kB
Step 1/18 : FROM continuumio/miniconda3:4.8.2
 ---> b4adc22212f1
Step 2/18 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 53fb2c825bc1
Step 3/18 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> b599776a4924
Step 4/18 : COPY . /bento
 ---> 65503dd2a391
Step 5/18 : WORKDIR /bento
 ---> Running in 19657538cf21
Removing intermediate container 19657538cf21
 ---> a599f651b380
Step 6/18 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in 86863fb4c7a1
Removing intermediate container 86863fb4c7a1
 ---> 7cb562709b0d
Step 7/18 : RUN conda env update -n base -f /bento/environment.yml
 ---> Running in 35b168429db1
Solving environment: ...working... done

chardet-3.0.4        | 180 KB    | ########## | 100% 
pysocks-1.7.1        | 

# Test with requests

In [15]:
import json
import requests
import pandas as pd


headers = {"content-type": "application/json"}
data = pd.DataFrame([[0.0, 2.0]]).to_json()
print(data)

json_response = requests.post(server_url,
                              data=data, headers=headers)
print(json_response)
print(json_response.json())

{"0":{"0":0.0},"1":{"0":2.0}}
<Response [200]>
[{'0': 0.0, '1': 2.0}]


# Benchmark

In [8]:
import random
import ujson as json

def get_request_producer(A, B):

    headers = {"content-type": "application/json"}

    def _gen_data():
        _A = A + random.random() / 100000.0
        _B = B + random.random() / 10000.0
        data = pd.DataFrame([[_A, _B]]).to_json()
        return server_url, "POST", headers, data

    return _gen_data

get_request_producer(A=0.2, B=0.5)()

('http://127.0.0.1:51793/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"0":{"0":0.2000056187},"1":{"0":0.5000152463}}')

In [31]:
A = 0.001
B = 0.001
pressure = 5

# benchmark under a proper amount of users (five times of the theoretical capacity)
proper_user_num = min(max(int(5 / A), 1), 900)
print("test user amount: ", proper_user_num)

from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(A=A, B=B), lambda: 1, timeout=11)
b.start_session(60, proper_user_num, proper_user_num * 1)

test user amount:  900
======= Session started! =======

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧══════════╧═════════════════╧═══════════════════╛

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │     667 │    333.5 │          0.4777 │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │      0   │        nan      │                   │
╘═════

In [32]:
def log_result(b, name, file_name, pressure):
    self = b.stat

    result = []
    result.append(name)
    result.append(self.success)
    result.append(self.success / max(self.sess_time, 1))
    result.append(sum(self.succ_times) / max(self.success, 1))

    result.append(self.fail)
    result.append(self.fail / max(self.sess_time, 1))
    result.append(sum(self.exec_times) / max(self.fail, 1))
    log_str = ','.join(str(r) for r in result)
    with open(file_name, "a") as lf:
        lf.write(log_str)
        lf.write('\n')


log_result(b, f'{NAME}_{A}_{B}', 'benchmark_result.csv')